In [1]:
%%capture
!pip install -q dagshub mlflow python-dotenv langchain langchain-openai langchain-together langchain-community pypdf pinecone-client[grpc] langchain-pinecone langchain-huggingface

In [2]:
#@markdown **You need to sign up for [DagsHub](https://dagshub.com/user/sign_up) , then enter the name of the repository you'd like to create, and your username and email.**

#@markdown Enter the repository name for the project:
REPO_NAME= "komrag" #@param {type:"string"}

#@markdown Enter the username of your DAGsHub account:
USER_NAME = "OsipovStas" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
EMAIL = "stasstels@gmail.com" #@param {type:"string"}

#@markdown ---

import IPython

In [3]:
from google.colab import userdata
DH_TOKEN = userdata.get('LLM_MATH_COMP_DH_TOKEN')


In [4]:
import mlflow
import dagshub
import os
#DH_TOKEN = os.environ['LLM_MATH_COMP_DH_TOKEN']
username = f'{USER_NAME}'  # Replace with your DagsHub username
repository = f'{REPO_NAME}'  # Replace with your repository name
token = f'{DH_TOKEN}'  # Replace with your DagsHub token
os.environ['MLFLOW_TRACKING_USERNAME'] = username
os.environ['MLFLOW_TRACKING_PASSWORD'] = token
dagshub.auth.add_app_token(token)
dagshub.init(repo_name=REPO_NAME, repo_owner=USER_NAME)
mlflow.set_tracking_uri(f"https://dagshub.com/{USER_NAME}/{REPO_NAME}.mlflow")

Accessing as OsipovStas

Initialized MLflow to track repo "OsipovStas/komrag"

Repository OsipovStas/komrag initialized!

In [5]:
os.environ['LANGCHAIN_API_KEY'] = userdata.get("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = "komrag"

In [ ]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("OsipovStas/komrag")

# Upload file
s3.upload_file(
    Bucket="komrag",  # name of the repo
    Filename="./life_begin.pdf",  # local path of file to upload
    Key="life_begin.pdf",  # remote path where to upload the file
)

Client created. Use the name of the repo (komrag) as the name of the bucket

In [ ]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("OsipovStas/komrag")


s3.download_file(
    Bucket="komrag",  # name of the repo
    Key="life_begin.pdf",  #  remote path from where to download the file
    Filename="life_begin.pdf",  # local path where to download the file
)

Client created. Use the name of the repo (komrag) as the name of the bucket

In [ ]:
from dagshub.notebook import save_notebook

save_notebook(repo=f"{USER_NAME}/{REPO_NAME}", path="./notebooks/exploratory-colab.ipynb")

# LLM Configuration

In [ ]:
model="gpt-4o-mini-2024-07-18"

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(model=model, temperature=0, timeout=120)

In [ ]:
from langchain_together import ChatTogether

# choose from our 50+ models here: https://docs.together.ai/docs/inference-models
chat = ChatTogether(
    together_api_key=userdata.get("TOGETHER_KEY"),
    model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
    timeout=120
)

In [ ]:
chat.invoke("Hello, do you know russian? Write a response in it")

AIMessage(content='Да, я знаю русский! (Yes, I know Russian!) \n\nВам что-нибудь говорят? (Do you want me to say something?)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 22, 'total_tokens': 55, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-81d9dd5b-fe7f-4da6-acff-172dc25c07d2-0', usage_metadata={'input_tokens': 22, 'output_tokens': 33, 'total_tokens': 55, 'input_token_details': {}, 'output_token_details': {}})

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=768, api_key=userdata.get("OPENAI-KEY"))

In [ ]:
from langchain_together import TogetherEmbeddings

embeddings = TogetherEmbeddings(
    together_api_key=userdata.get("TOGETHER_KEY"),
    model="togethercomputer/m2-bert-80M-8k-retrieval",
)



In [14]:
len(embeddings.embed_query("Приветы, как дела?"))

768

# Loading

In [ ]:
file_path = "life_begin.pdf"

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [ ]:
len(pages)

449

# Text cleanup

In [ ]:
template="""
You will receive a text in Russian extracted using an OCR tool.
Your task is to clean and format this text to enhance readability.
The text may contain various artifacts introduced during the OCR process, such as:

 - Special characters or symbols that do not belong.
 - Inconsistent casing of letters (e.g., random uppercase or lowercase letters).
 - Incorrect spacing within words or between words (e.g., words glued together or unnecessary spaces).
 - Hyphenation errors, such as words split across lines.
 - Incorrect punctuation or misplaced commas and periods.
 - New line characters

Please ensure the final output is free from these artifacts and is well-formatted.
Pay attention to punctuation, paragraph structure, and overall coherence.
The goal is to produce a clean, easy-to-read text.
Output ONLY the cleaned text.


### TEXT ###

{text}

"""

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = prompt | chat | StrOutputParser()

In [ ]:
print(chain.invoke({"text": pages[30].page_content}))

ДО БЕРЕМЕННОСТИ

Лечение и медицинские процедуры

1.3.1. ПРИВИВКИ. КАКИЕ, КОМУ, КОГДА?

Тема вакцинации обязательна к обсуждению с вашим семейным врачом. Надеюсь, вы привиты в соответствии с национальным календарем. «Вы» — это будущие родители, старшие дети, другие члены семьи.

О чем следует помнить и на что обратить внимание:

- Вакцинация от коклюша, дифтерии и столбняка. Последняя «детская» инъекция делается в 16 лет. После этого необходимо прививаться каждые 10 лет. Всем. Даже тем, кто не планирует беременность.
- Следует знать, что чуть ли не самая опасная инфекция для младенца — коклюш, который он может «получить» от ближайших непривитых родственников. Именно поэтому следует убедиться в том, что от коклюша привиты все те, кто будет контактировать с младенцем в первые 2 месяца его жизни (т. е. до того, как прививки начнут делать самому ребенку).
- Краснуха — чрезвычайно опасная инфекция именно для плода. Есть ли документальное подтверждение тому, что вы получили 2 дозы вакцины КП

In [ ]:
def clean_text(text):
    # try up to 3 times if no success return text as is
    for _ in range(3):
        try:
            return (chain.invoke({"text": text}), True)
        except Exception as e:
            print(f"Error cleaning text: {e}")
    return (text, False)

In [ ]:
from tqdm import tqdm

failures = []
for p in tqdm(pages, desc="Cleaning text", unit="page"):
  text, suc = clean_text(p.page_content)
  if not suc:
    failures.append(p)
  else:
    p.page_content = text

In [ ]:
print(pages[3].page_content)

# Splitting

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
splits = text_splitter.split_documents(pages)

len(splits)

In [ ]:
splits[8]

# Indexing

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = pinecone.Pinecone(api_key=os.environ['PINECONE_API_KEY'])

In [ ]:
from pinecone import Pinecone, ServerlessSpec


pc = pinecone.Pinecone(api_key=userdata.get('PINECONE_KEY'))

In [ ]:
pc.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'komrag-mcvbmc1.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'komrag',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [ ]:
import time

index_name = "komrag"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
pc.list_indexes()

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
import hashlib

def generate_id(content):
    # Ensure the content is in bytes
    if isinstance(content, str):
        content = content.encode('utf-8')

    # Create a SHA-256 hash object
    hash_object = hashlib.sha256()

    # Update the hash object with the content
    hash_object.update(content)

    # Get the hexadecimal representation of the hash
    unique_id = hash_object.hexdigest()

    return unique_id

In [ ]:
def index_all_docs(docs):
  # process docs by batches of 100 docs using tqdm
  for i in tqdm(range(0, len(docs), 100), desc="Indexing", unit="batch"):
    index_batch(docs[i:i+100])

def index_batch(docs):
  ids = [generate_id(doc.page_content) for doc in docs]
  vector_store.add_documents(documents=docs, ids=ids)

In [ ]:
index_all_docs(pages)